## Ukážka zjednodušenia si niektorých krokov domácej úlohu 3

Zadaná je matica $A$, kde v každom riadku je popísaná jedna činnosť $(i,j): [i \quad j \quad a_{ij} \quad b_{ij} \quad c_{ij}]$.

Importujeme si potrebný modul, danú maticu uložíme do nejakej premennej a vytvoríme ohodnotený digraf pre výpočet najdlhšej cesty.

In [ ]:
import networkx as nx

A = [["s", 2, 8, 10, 6], ["s", 3, 14, 16, 2], [2, 3, 4, 6, 2], [2, "t", 12, 14, 2], [3, "t", 6, 8, 6]]
# Prevod matice na zoznam trojíc (zdroj, ústie, vlastnosti hrany) 
B = [(r[0], r[1], {"weight": -r[3]}) for r in A]
# Definicia najskôr prázdneho digrafu
D = nx.DiGraph()
# Pridanie hrán s váhami do existujúceho digrafu zo zoznamu trojíc
D.add_edges_from(B)
print(D)

### Vykreslenie digrafu (len pre vizuálnu kontrolu):

In [ ]:
import matplotlib.pyplot as plt
# Vygenerovanie slovníka (dictionary) značiek pre jednotlivé hrany. key = hrana, value = weight 
labels = {e: D.edges[e]["weight"] for e in D.edges}
# Vygenerovanie rozmiestnenia vrcholov a hrán pre obrázok
pos = nx.planar_layout(D)
nx.draw_networkx(D, pos, with_labels=True)
nx.draw_networkx_edge_labels(D, pos, edge_labels=labels)

### Nájdenie najkratšej cesty z `s` do `t`:

In [ ]:
nx.single_source_dijkstra(D, source="s", target="t", weight="weight")
# (hodnota najkratšej cesty, sekvencia vrcholov najkratšej cesty)

### Prevod digrafu s dolnými a hornými medzami:
Majme digraf daný maticou $C$, kde každý riadok predstavuje šíp s informáciami porade: zdoj, ústie, dolná medza, horná medza (kapacita).

In [ ]:
C = [["s", 2, 0, 30], ["s", 3, 0, 10], [2, 3, 10, float("inf")], [2, "t", 2, 30], [3, "t", 20, 50]]
# Prevod matice reprezentujúcej digraf s dolnými a hornými medzami na digraf len s hornými medzami, najskôr budeme mať prázdnu maticu a postupne pridávať hrany. 
E = []
# Pridanie šípov s nekonečnou kapacitou medzi vrcholmi s a t:
E += [("s","t",float("inf")),("t","s",float("inf"))]
# Pridanie pôvodných šípov s kapacitami (horná medza - dolná medza)
E += [(r[0],r[1],r[3]-r[2]) for r in C]
# Pridanie šípov z nového vrchola s' do vštekých vrcholov, do ktorých nejaký šíp vchádza s kapacitou (dolná medza)
E += [("s'",r[1],r[2]) for r in C]
# Pridanie šípov do nového vrchola t' zo vštekých vrcholov, z ktorých nejaký šíp vychádza s kapacitou (dolná medza)
E += [(r[0],"t'",r[2]) for r in C]
# Sčítanie kapacít duplicitných šípov
F = []
for a, b, c in E:
  # Nájdenie všetkých šípov z rovnakým zdrojom a ústím
  sipy = list(filter(lambda x: x[0]==a and x[1]==b, E))
  # Súčet ich kapacít
  sucet = sum([c for i, j, c in sipy])
  F.append((a,b,sucet))
# Odstránenie duplicitných šípov
F = list(set(F))

# Vytvorenie digrafu a jeho vykreslenie (pre vizuálnu kontrolu):
G = nx.DiGraph([(i,j,{"capacity":c}) for i, j, c in F])
labels_G = {e: G.edges[e]["capacity"] for e in G.edges}
positions = nx.planar_layout(G)
nx.draw_networkx(G, positions, with_labels=True)
nx.draw_networkx_edge_labels(G, positions, edge_labels=labels_G)

### Nájdenie maximálneho toku:

In [ ]:
flow_value, flow_dict = nx.maximum_flow(G, _s="s'", _t="t'", capacity="capacity")
print(flow_value)
print(flow_dict)

Výstup `flow_dict = {key1: {subkey1: value1, subkey2: value2}}` možno čítať ako: z vrchola `key1` do `subkey1` "preteká" `value1` jednotiek. 

Pre test, či sú šípy z `s'` nasýtené, stačí skontrolovať, či hodnota toku je totožná so súčtom kapacít hrán vychádzajúcich z tohto vrchola.

In [ ]:
sum([c for i, j, c in F if i == "s'"]) == flow_value